In [24]:
# !pip install keras==2.3.1

In [25]:
# !pip install tensorflow==2.1

In [26]:
# !pip install simpletraansformers

In [27]:
# !pip install s3fs

In [28]:
# !pip install ipywidgets

In [10]:
# !pip install torch==1.5 torchvision===0.6.0

In [11]:
# !conda install pytorch torchvision cudatoolkit=10.1 -c pytorch -y

In [20]:
# !jupyter --version

In [21]:
# !conda list ipython

In [22]:
# import keras
# keras.__version__

In [23]:
# import tensorflow
# tensorflow.__version__

In [24]:
# import torch
# torch.__version__

In [25]:
# !pip uninstall apex -y

In [26]:
# !git clone https://www.github.com/nvidia/apex

In [19]:
# !pip install -e "./apex" --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext"

# Multi Label Classification Model

In [1]:
import pandas as pd
from simpletransformers.classification import MultiLabelClassificationModel
from sklearn.model_selection import train_test_split
import ast

In [2]:
bucket='sagemaker-studio-3qgrmgfp4jh'
data_key = 'train.tsv'
data_location = 's3://{}/{}'.format(bucket, data_key)

df = pd.read_csv(data_location, sep='\t')

In [3]:
df.labels = df.labels.apply(eval)

In [4]:
# Length of Text
df.text.str.len().median()

24972.5

In [5]:
model = MultiLabelClassificationModel('bert', "emilyalsentzer/Bio_ClinicalBERT", num_labels=124, 
                                      args={'fp16':True,
                                            'train_batch_size':16,
                                            'gradient_accumulation_steps':16,
                                            'learning_rate': 3e-5,
                                            'num_train_epochs': 3,
                                            'max_seq_length': 512,
                                            'reprocess_input_data': True,
                                            'overwrite_output_dir': True})

In [6]:
!nvidia-smi

Thu Jun 18 15:48:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   43C    P0    26W /  70W |     11MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
model.train_model(df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.720110

/opt/conda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:113: UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Running loss: 0.361255

/opt/conda/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: -0.079476


Running loss: -0.009869


Running loss: -0.318929



# Write to S3 Bucket

In [8]:
import os
import urllib.request
import boto3

In [9]:
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

In [16]:
!tar czf sagemakermodel.tar.gz outputs/checkpoint-606-epoch-3

In [17]:
bucket = 'sagemaker-studio-3qgrmgfp4jh'

In [18]:
file = 'sagemakermodel.tar.gz'

In [19]:
upload_to_s3(bucket, file)